In [4]:
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
from citeline.database.milvusdb import MilvusDB
from citeline.embedders import Embedder

tqdm.pandas()

db = MilvusDB()
print(db)
db.list_collections()

Collections:
 - astrobert_chunks: 460801 entities
 - astrobert_contributions: 89860 entities
 - bge_chunks: 460801 entities
 - bge_contributions: 89860 entities
 - nasa_chunks: 460801 entities
 - nasa_contributions: 89860 entities
 - qwen06_chunks: 460801 entities
 - qwen06_contributions: 89860 entities
 - qwen8b_contributions: 89860 entities
 - specter_chunks: 460801 entities
 - specter_contributions: 89860 entities


This notebook takes a jsonl file of 'findings' with schema `{"findings": [str], "doi": str}` and preps a dataset for insertion into Milvus. This requires 
1. From the research data used, join to each findings line the:
- citation_count
- pubdate (YYYYMMDD int)
2. Explode the dataframe along findings to get one row per finding
3. Invoke `db.create_collection` with the desired embedder

In [5]:
research = pd.read_json("../data/research_used.jsonl", lines=True)
# add index to 'doi' column
research.set_index('doi', inplace=True, verify_integrity=True, drop=False)